### Download, and unzip yelp academic dataset for inspection

The Yelp Academic Dataset was download from https://www.yelp.com/dataset/challenge and moved into the project project folder.

In [1]:
!python --version 

Python 3.6.3 :: Anaconda, Inc.


In [2]:
import os
import sys
import tarfile
import json
from sqlalchemy import create_engine
import psycopg2
import pandas as pd

sys.path.append('../')
from envir import config

In [3]:
if not os.path.isdir(config.shared + 'yelp_dataset/'):
    print('new directry has been created')
    os.system('mkdir yelp_dataset')
    tar = tarfile.open(config.shared + 'yelp_dataset.tar', mode = 'r')
    tar.extractall(config.shared + 'yelp_dataset/')

In [4]:
yelpFolder = config.shared + 'yelp_dataset/'

In [5]:
data = []
with open(yelpFolder+'business.json') as f:
    for line in f:
        data.append(json.loads(line))

In [6]:
data[0]

{'business_id': '1SWheh84yJXfytovILXOAQ',
 'name': 'Arizona Biltmore Golf Club',
 'address': '2818 E Camino Acequia Drive',
 'city': 'Phoenix',
 'state': 'AZ',
 'postal_code': '85016',
 'latitude': 33.5221425,
 'longitude': -112.0184807,
 'stars': 3.0,
 'review_count': 5,
 'is_open': 0,
 'attributes': {'GoodForKids': 'False'},
 'categories': 'Golf, Active Life',
 'hours': None}

In [7]:
yelp = pd.DataFrame.from_dict(data)

In [20]:
yelp.groupby('city').agg(['count'])

,business_id,name,address,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
,count,count,count,count,count,count,count,count,count,count,count,count,count
city,,,,,,,,,,,,,
,1,1,1,1,1,1,1,1,1,1,1,1,1
110 Las Vegas,1,1,1,1,1,1,1,1,1,1,1,1,1
AGINCOURT,1,1,1,1,1,1,1,1,1,1,1,1,1
AVON,1,1,1,1,1,1,1,1,1,1,1,1,1
AZ,1,1,1,1,1,1,1,1,1,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
toronto,3,3,3,3,3,3,3,3,3,3,3,3,3
verdun,1,1,1,1,1,1,1,1,1,1,1,1,1


In [8]:
yelp[yelp['state']=='NY'].groupby('city').agg(['count'])

,business_id,name,address,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
,count,count,count,count,count,count,count,count,count,count,count,count,count
city,,,,,,,,,,,,,
Brooklyn,1,1,1,1,1,1,1,1,1,1,1,1,1
Champlain,10,10,10,10,10,10,10,10,10,10,10,10,7
Closter,1,1,1,1,1,1,1,1,1,1,1,1,1
Mooers,1,1,1,1,1,1,1,1,1,1,1,1,1
New York,1,1,1,1,1,1,1,1,1,1,1,1,1
North Las Vegas,1,1,1,1,1,1,1,1,1,1,1,1,1
Rouses Point,7,7,7,7,7,7,7,7,7,7,6,7,5


The Yelp Academic dataset did not have many New York ratings. Will have to use the API

### checking other cities

In [9]:
yelp.groupby('city').count().reset_index().sort_values('business_id',ascending=False).head(20)

,city,business_id,name,address,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
451,Las Vegas,29370,29370,29370,29370,29370,29370,29370,29370,29370,29370,25043,29294,22774
1066,Toronto,18906,18906,18906,18906,18906,18906,18906,18906,18906,18906,16140,18876,13883
780,Phoenix,18766,18766,18766,18766,18766,18766,18766,18766,18766,18766,15697,18699,14582
163,Charlotte,9509,9509,9509,9509,9509,9509,9509,9509,9509,9509,8165,9489,7391
959,Scottsdale,8837,8837,8837,8837,8837,8837,8837,8837,8837,8837,7581,8824,7196
133,Calgary,7736,7736,7736,7736,7736,7736,7736,7736,7736,7736,6199,7723,5685
801,Pittsburgh,7017,7017,7017,7017,7017,7017,7017,7017,7017,7017,6107,7008,5217
616,Montréal,6449,6449,6449,6449,6449,6449,6449,6449,6449,6449,5831,6433,4718
568,Mesa,6080,6080,6080,6080,6080,6080,6080,6080,6080,6080,5147,6062,4885
371,Henderson,4892,4892,4892,4892,4892,4892,4892,4892,4892,4892,4167,4877,3902


In [10]:
yelp[yelp['categories'].str.contains('Restaurants')==True][yelp['city']=='Las Vegas'].count()

/home/cusp/yg833/.conda/envs/yelpVE/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


business_id     6450
name            6450
address         6450
city            6450
state           6450
postal_code     6450
latitude        6450
longitude       6450
stars           6450
review_count    6450
is_open         6450
attributes      6294
categories      6450
hours           5004
dtype: int64

In [11]:
yelp[yelp['categories'].str.contains('Restaurants')==True][yelp['city']=='Toronto'].count()

/home/cusp/yg833/.conda/envs/yelpVE/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


business_id     7964
name            7964
address         7964
city            7964
state           7964
postal_code     7964
latitude        7964
longitude       7964
stars           7964
review_count    7964
is_open         7964
attributes      7614
categories      7964
hours           6064
dtype: int64

In [12]:
yelp[yelp['categories'].str.contains('Restaurants')==True][yelp['city']=='Phoenix'].count()

/home/cusp/yg833/.conda/envs/yelpVE/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


business_id     3999
name            3999
address         3999
city            3999
state           3999
postal_code     3999
latitude        3999
longitude       3999
stars           3999
review_count    3999
is_open         3999
attributes      3907
categories      3999
hours           3005
dtype: int64

In [13]:
yelp.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,1SWheh84yJXfytovILXOAQ,Arizona Biltmore Golf Club,2818 E Camino Acequia Drive,Phoenix,AZ,85016,33.522143,-112.018481,3.0,5,0,{'GoodForKids': 'False'},"Golf, Active Life",None
1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,L5R 3E7,43.605499,-79.652289,2.5,128,1,"{'RestaurantsReservations': 'True', 'GoodForMe...","Specialty Food, Restaurants, Dim Sum, Imported...","{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W..."
2,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,"10110 Johnston Rd, Ste 15",Charlotte,NC,28210,35.092564,-80.859132,4.0,170,1,"{'GoodForKids': 'True', 'NoiseLevel': 'u'avera...","Sushi Bars, Restaurants, Japanese","{'Monday': '17:30-21:30', 'Wednesday': '17:30-..."
3,xvX2CttrVhyG2z1dFg_0xw,Farmers Insurance - Paul Lorenz,"15655 W Roosevelt St, Ste 237",Goodyear,AZ,85338,33.455613,-112.395596,5.0,3,1,None,"Insurance, Financial Services","{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ..."
4,HhyxOkGAM07SRYtlQ4wMFQ,Queen City Plumbing,"4209 Stuart Andrew Blvd, Ste F",Charlotte,NC,28217,35.190012,-80.887223,4.0,4,1,"{'BusinessAcceptsBitcoin': 'False', 'ByAppoint...","Plumbing, Shopping, Local Services, Home Servi...","{'Monday': '7:0-23:0', 'Tuesday': '7:0-23:0', ..."


In [14]:
# conversion function:
def dict2json(dictionary):
    return json.dumps(dictionary, ensure_ascii=False)

yelp['attributes'] = yelp['attributes'].map(dict2json)
yelp['hours'] = yelp['hours'].map(dict2json)

### send to db

In [15]:
engine = create_engine('postgresql+psycopg2:///yelp_abcs')
con = engine.connect()
try:
    yelp.to_sql('yelp_academic_business', engine, schema='stag')
except ValueError:
    print('Table Already Exists')
    

Table Already Exists


### get reviews data

In [16]:
data = []
with open(yelpFolder+'review.json') as f:
    for line in f:
        data.append(json.loads(line))
        
data[0]

{'review_id': 'Q1sbwvVQXV2734tPgoKj4Q',
 'user_id': 'hG7b0MtEbXx5QzbzE6C_VA',
 'business_id': 'ujmEBvifdJM6h6RLv4wQIg',
 'stars': 1.0,
 'useful': 6,
 'funny': 1,
 'cool': 0,
 'text': 'Total bill for this horrible service? Over $8Gs. These crooks actually had the nerve to charge us $69 for 3 pills. I checked online the pills can be had for 19 cents EACH! Avoid Hospital ERs at all costs.',
 'date': '2013-05-07 04:34:36'}

In [17]:
yelpReview = pd.DataFrame.from_dict(data)

In [18]:
yelpReview.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,Q1sbwvVQXV2734tPgoKj4Q,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg,1.0,6,1,0,Total bill for this horrible service? Over $8G...,2013-05-07 04:34:36
1,GJXCdrto3ASJOqKeVWPi6Q,yXQM5uF2jS6es16SJzNHfg,NZnhc2sEQy3RmzKTZnqtwQ,5.0,0,0,0,I *adore* Travis at the Hard Rock's new Kelly ...,2017-01-14 21:30:33
2,2TzJjDVDEuAW6MR5Vuc1ug,n6-Gk65cPZL6Uz8qRm3NYw,WTqjgwHlXbSFevF32_DJVw,5.0,3,0,0,I have to say that this office really has it t...,2016-11-09 20:09:03
3,yi0R0Ugj_xUx_Nek0-_Qig,dacAIZ6fTM6mqwW5uxkskg,ikCg8xy5JIg_NGPx-MSIDA,5.0,0,0,0,Went in for a lunch. Steak sandwich was delici...,2018-01-09 20:56:38
4,11a8sVPMUFtaC7_ABRkmtw,ssoyf2_x0EQMed6fgHeMyQ,b1b1eb3uo-w561D0ZfCEiQ,1.0,7,0,0,Today was my second out of three sessions I ha...,2018-01-30 23:07:38


In [19]:
try:
    yelpReview.to_sql('yelp_academic_review', engine, schema='stag')
except:
    print('table exists in db')

table exists in db
